In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpim
import numpy as np
import random
import cv2

training_path = "fruits-360_dataset/Training"
test_path = "fruits-360_dataset/Test"

try:
    STATS = np.load("stats.npy", allow_pickle=True)
except FileNotFoundError as fnf:
    print("Not found stats file.")
    STATS = []

# Parameters    
GRAY_SCALE = False
FRUITS = os.listdir(training_path)

random.shuffle(FRUITS)
FRUITS = FRUITS[:15] 


Not found stats file.


In [2]:
def load_data(directory_path, fruit_restriction=None):
    data = []
    labels = []
               
            
    for fruit_name in FRUITS:
        class_num = FRUITS.index(fruit_name)                
        
        path = os.path.join(directory_path, fruit_name)
        print(f"Adding: {class_num:>3} {fruit_name:<20}, {path}")
        
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            
            if GRAY_SCALE:
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            else:
                image = mpim.imread(img_path)
                
            image = image / 255.0
                    
            data.append([image, class_num])

#         print(FRUITS[class_num])
#         plt.figure()    
#         plt.imshow(image, cmap='gray')
#         plt.show()
        
    random.shuffle(data)
        
    X = []
    y = []
    
    
    for image, label in data:
        X.append(image)
        y.append(label)
    
#     X = data[:, 0]
#     y = data[:, 1]
    X = np.array(X)
    y = np.array(y)
    
    if GRAY_SCALE:
        print("Reshaping gray scale")
        X = X.reshape(-1, X.shape[1], X.shape[2], 1)
        
    return X, y      

In [3]:
X_training, y_training = load_data(training_path)
print("\nCreated training array")    
print(f"X shape: {X_training.shape}")
print(f"y shape: {y_training.shape}")

Adding:   0 Potato Sweet        , fruits-360_dataset/Training/Potato Sweet
Adding:   1 Walnut              , fruits-360_dataset/Training/Walnut
Adding:   2 Nut Pecan           , fruits-360_dataset/Training/Nut Pecan
Adding:   3 Cherry Wax Yellow   , fruits-360_dataset/Training/Cherry Wax Yellow
Adding:   4 Grape White 3       , fruits-360_dataset/Training/Grape White 3
Adding:   5 Lemon Meyer         , fruits-360_dataset/Training/Lemon Meyer
Adding:   6 Cantaloupe 1        , fruits-360_dataset/Training/Cantaloupe 1
Adding:   7 Maracuja            , fruits-360_dataset/Training/Maracuja
Adding:   8 Apple Red 1         , fruits-360_dataset/Training/Apple Red 1
Adding:   9 Orange              , fruits-360_dataset/Training/Orange
Adding:  10 Potato Red          , fruits-360_dataset/Training/Potato Red
Adding:  11 Potato White        , fruits-360_dataset/Training/Potato White
Adding:  12 Avocado             , fruits-360_dataset/Training/Avocado
Adding:  13 Papaya              , fruits-360_da

In [4]:
X_test, y_test = load_data(test_path)
print("\nCreated test array")    
print(f"X shape: {X_test.shape}")
print(f"y shape: {y_test.shape}")

Adding:   0 Potato Sweet        , fruits-360_dataset/Test/Potato Sweet
Adding:   1 Walnut              , fruits-360_dataset/Test/Walnut
Adding:   2 Nut Pecan           , fruits-360_dataset/Test/Nut Pecan
Adding:   3 Cherry Wax Yellow   , fruits-360_dataset/Test/Cherry Wax Yellow
Adding:   4 Grape White 3       , fruits-360_dataset/Test/Grape White 3
Adding:   5 Lemon Meyer         , fruits-360_dataset/Test/Lemon Meyer
Adding:   6 Cantaloupe 1        , fruits-360_dataset/Test/Cantaloupe 1
Adding:   7 Maracuja            , fruits-360_dataset/Test/Maracuja
Adding:   8 Apple Red 1         , fruits-360_dataset/Test/Apple Red 1
Adding:   9 Orange              , fruits-360_dataset/Test/Orange
Adding:  10 Potato Red          , fruits-360_dataset/Test/Potato Red
Adding:  11 Potato White        , fruits-360_dataset/Test/Potato White
Adding:  12 Avocado             , fruits-360_dataset/Test/Avocado
Adding:  13 Papaya              , fruits-360_dataset/Test/Papaya
Adding:  14 Peach               , 

In [5]:
# Limit gpu memory usage
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.2
sess = tf.compat.v1.Session(config=config)

In [6]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, Activation, Dropout

pic_shape = X_training.shape[1:]
label_count = len(FRUITS)

print(f"Pic shape: {pic_shape}")

# Input
model = tf.keras.models.Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=pic_shape),
    MaxPooling2D(pool_size=(2, 2)),
    
    Dropout(0.2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(32, activation='relu'),
    Dense(label_count, activation='softmax')    
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Pic shape: (100, 100, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 33856)             0         
_________________________________________________________________
dense (Dense)                (N

In [7]:
model.fit(X_training, y_training, batch_size=20, epochs=3)

Train on 7101 samples
Epoch 1/3
7101/7101 [==============================] - 8s 1ms/sample - loss: 0.5332 - accuracy: 0.8238
Epoch 2/3
7101/7101 [==============================] - 5s 679us/sample - loss: 0.0248 - accuracy: 0.9923
Epoch 3/3
7101/7101 [==============================] - 5s 674us/sample - loss: 0.0650 - accuracy: 0.9827


In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
predictions = model.predict(X_test)

print(f"Test Accuracy: {accuracy:>2.4f}")
print(f"Test loss: {loss:>2.4f}")

count_incorrect = 0
for x, predict in enumerate(predictions):
    class_num = np.argmax(predict)
    valid = y_test[x] == class_num
    if not valid:
        count_incorrect += 1
print(f"Incorrect predictions: {count_incorrect}")       


2496/2496 [==============================] - 1s 253us/sample - loss: 0.2680 - accuracy: 0.8986
Test Accuracy: 0.8986
Test loss: 0.2680
Incorrect predictions: 253
